### Lattice class -> intialization, algorithms on it

In [4]:
import random
import copy
import itertools
import numpy as np
import pandas as pd

class lattice:
    
    def __init__(self,L,p):
        
        self.L=L
        self.p=p
        self.lattice=[[-1]*(self.L+2)] #laticce sourronded by -1
        #STUFF FOR BURNING METHOD:
        self.reached_last_row=False
        self.no_t_cell=False #there are no more cells with t number, burning method
        #STUFF FO HOSHEN:
        self.labels=[0,0]
        self.labels.extend([x for x in range(2,self.L*self.L)]) #labels[i]=i kazdy na poczatku nie jest z nikim zwiazany oprocz siebie
        self.counters=[0]*(self.L*self.L)
        #CREATE LATTICE - RANDOM
        for i in range(0,self.L):
            lattice_row=[-1]
            for j in range(0,self.L):
                r=random.random()
                if self.p>r:
                    lattice_row.append(1)
                else:
                    lattice_row.append(0)
                    
            lattice_row.append(-1)
            self.lattice.append(lattice_row)
            
        self.lattice.append([-1]*(self.L+2))
        self.lattice_copy=copy.deepcopy(self.lattice)
    
    def set_lattice(self, table):
        self.lattice=table
    
    def restart_lattice(self): #ONCE BUNRING METHOD PERFORM, RESTART LATTICE TO GET NORMAL TABLE AND PEROFORM HASHEN ON IT
        #PARAMETERS RESTART:
        self.reached_last_row=False #burning method
        self.no_t_cell=False #there are no more cells with t number, burning method
        self.labels=[0,0]
        self.labels.extend([x for x in range(2,self.L*self.L)]) 
        self.counters=[0]*(self.L*self.L)
        #LATTICE RESTART:
        self.lattice=copy.deepcopy(self.lattice_copy)
        
    def new_random_lattice(self): #new random lattice
        
        self.lattice=[[-1]*(self.L+2)]
        self.reached_last_row=False
        self.no_t_cell=False 
        self.labels=[0,0]
        self.labels.extend([x for x in range(2,self.L*self.L)]) 
        self.counters=[0]*(self.L*self.L)
        
        for i in range(0,self.L):
            
            lattice_row=[-1]
            
            for j in range(0,self.L):
                r=random.random()
                if self.p>r:
                    lattice_row.append(1)
                else:
                    lattice_row.append(0)
                    
            lattice_row.append(-1)
            self.lattice.append(lattice_row)
            
        self.lattice.append([-1]*(self.L+2))
        self.lattice_copy=copy.deepcopy(self.lattice)
    
    def print_lattice(self):
        for i in self.lattice:
            print(i)

#BURNING METHOD FUNCTIONS: 
    def check_neighbours(self,t,i,j): #burning method
        
        if self.lattice[i][j+1]==1:
            self.lattice[i][j+1]=t+1
            
        if self.lattice[i][j-1]==1:
            self.lattice[i][j-1]=t+1
            
        if self.lattice[i+1][j]==1:
            self.lattice[i+1][j]=t+1
            
        if self.lattice[i-1][j]==1:
            self.lattice[i-1][j]=t+1
    
    def iteration(self,t): #burning method
        self.no_t_cell=True
        for i in range(0,len(self.lattice)):
            for j in range(0,len(self.lattice[0])):
                if self.lattice[i][j]==t:
                    self.no_t_cell=False
                    self.check_neighbours(t,i,j)
        #self.print_lattice()
        
    def reached_last_row_update(self): #burning method
        self.reached_last_row=False
        for i in self.lattice[-2]:
            if i>1:
                self.reached_last_row=True #dotarliśmy do ostatniego rzędu
            
    def burning_method(self): #shortest path finding
        t=2
        self.lattice[1]=[2 if i==1 else i for i in self.lattice[1]] #setting 2 in the first line
        #self.print_lattice()
        warunek=True
        while self.reached_last_row==False and self.no_t_cell==False:
            self.iteration(t)
            self.reached_last_row_update()
            t+=1
        try:
            return min([i for i in self.lattice[-2] if i>1])-1 #jest najkrotsza sciezka
        except:
            return 0
        
    def run_monte_burning_method(self):
        self.new_random_lattice() #new random lattice
        a=self.burning_method() #find shorest path
        return a
    
#HASHEN ALGORITHM FUNCTIONS:
    def find(self,x):
        while self.labels[x] != x:
            x = self.labels[x];
        return x;
    
    def union(self,x,y): 
        if self.find(y)==self.find(x):
            pass
        ### nowe
        else:
            self.counters[self.find(y)]+=self.counters[self.find(x)]
            self.counters[self.find(x)]=0
        
            self.labels[self.find(x)] = self.find(y);
        
    def hoshen_kopelman(self):
        self.restart_lattice() #DELETE EFFECT OF BURNING ALGORITHM
        k=1
        for i, j in itertools.product(range(len(self.lattice)), range(len(self.lattice))):
            top=self.lattice[i-1][j]
            left=self.lattice[i][j-1]
            if self.lattice[i][j]>0: #cell occupied
                #case 1 top and left empty we create a new cluster label and set it to the current cell
                if top<1 and left<1: 
                    k+=1
                    self.lattice[i][j]=k
                    self.counters[k]+=1
                # case 2 top ocuupied left empty we set the cluster label of the occupied cell to the current cell
                elif top>0 and left<1:
                    k0=self.find(top)
                    self.lattice[i][j]=k0
                    self.counters[k0]+=1
                # case 3 top empty left occupied
                elif top<1 and left>0:
                    k0=self.find(left)
                    self.lattice[i][j]=k0
                    self.counters[k0]+=1
                #case 4 both occpuied same k0
                elif top==left and top>0:
                    k0=self.find(left)
                    self.lattice[i][j] = k0#self.find(left)
                    self.counters[k0]+=1
                # case 5 both occupied different k0    
                else:
                    self.union(left,top)    
                    k0=self.find(left)
                    self.lattice[i][j] = k0#self.find(left)
                    self.counters[k0]+=1
                    
        return [i for i in self.counters if i>0]



### Checking if the code works

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
l=lattice(10,0.7)
fig, ax = plt.subplots(figsize=(7,7))
sns.heatmap(l.lattice, square=True, ax=ax,cmap="Blues")
najkrotsza_sciezka=l.burning_method()
print('najkrotsza sciezka',najkrotsza_sciezka)
clusters=l.hoshen_kopelman()
print('clusters',clusters)

najkrotsza sciezka 11
clusters [18, 2, 51]


### Read input

In [5]:
def read_input():
    input_data = open("perc_init.txt","r")
    data=input_data.readlines()
    L=data[0]
    L=L.split(" ")
    L=L[1].split('\n')
    L=int(L[0])

    T=data[1]
    T=T.split(" ")
    T=T[1].split('\n')
    T=int(T[0])

    p0=data[2]
    p0=p0.split(" ")
    p0=p0[1].split('\n')
    p0=float(p0[0])

    pk=data[3]
    pk=pk.split(" ")
    pk=pk[1].split('\n')
    pk=float(pk[0])

    dp=data[4]
    dp=dp.split(" ")
    dp=dp[1].split('\n')
    dp=float(dp[0])
    print('L',L,'T',T,'p0',p0,'pk',pk,'dp',dp)
    return L,T,p0,pk,dp

In [19]:
def generate_data(T,p,L): #data for a specific probability, T trials
    print("p",p)
    s_max=[]
    s_ave=[]
    path=[]
    n_s_p_l=[]
    l=lattice(L,p)
    
    for i in range(0,T):
        #l=lattice(L,p)
        l.new_random_lattice()
        #print(l.print_lattice())
        najkrotsza_sciezka=l.burning_method()
        path.append(najkrotsza_sciezka)
        clusters=l.hoshen_kopelman()
        #print('clusters',clusters)
        try: #WHAT IF CLUSTERS ARE EMPTY- LOW PROBABILITY OF 1 ASSIGNING THEN MAX WON'T WORK?
            s_max.append(max(clusters))
            s_ave.append(sum(clusters)/len(clusters))
        except: #no clusters localized, clusters list is empty
            s_max.append(0)
            s_ave.append(0)
        n_s_p_l+=clusters  #duza lista z rozkladami clustrow, tzn lista z wielkosciami clustrow w T powtorzen
        #print('n_s_p_l',n_s_p_l)
    
    s_max_average=sum(s_max)/len(s_max)
    s_average=sum(s_ave)/len(s_ave)
    cluster_size,how_many_times=np.histogram(n_s_p_l,bins=2500)
    average_shortest_path=sum(path)/len(path)
    pflow=len([1 for i in path if i>0])/T
    #print('n_s_p_l','p',p,n_s_p_l)
    #print('resulting size, frequency',cluster_size,how_many_times)
    return s_max_average, s_average, average_shortest_path, pflow, cluster_size, how_many_times

In [20]:
def generate_file():
    L,T,p0,pk,dp=read_input()
    #print('T',T,'p0',p0,'pk',pk,'dp',dp,'L',L)
    s_max_list=[]
    s_ave_list=[]
    path_list=[]
    pflow_list=[]
    ps=np.arange(p0,pk,dp)
    
    for p in range(int(p0*10000),int(pk*10000),int(dp*10000)):
        
        s_max_average, s_average, average_shortest_path, pflow, cluster_size, how_many_times = generate_data(T,p/10000,L)
        s_max_list.append(s_max_average)
        s_ave_list.append(s_average)
        path_list.append(average_shortest_path)
        pflow_list.append(pflow)
        
        #n(s,p,L) file generation for each probability
        df2={'size':cluster_size, 'n(s,p,L)': how_many_times[:-1]}
        df=pd.DataFrame(df2)
        distribution_of_clusters_file='Dist_p'+str(p/10000)+'L'+str(L)+'T'+str(T)+'.txt'
        np.savetxt('C:/Users/wojci/Documents/Studia/4 rok/Complex systems/Raport 1/generated_data_files/clusters_dist/'+
                   distribution_of_clusters_file,
                   df.values, fmt='%1.2f')
   
    df={'p':ps,'pflow':pflow_list,'d':path_list,'smax':s_max_list,'save':s_ave_list}
    df=pd.DataFrame(df)
    plik="Ave_L"+str(L)+'T'+str(T)+'.txt'
    np.savetxt('C:/Users/wojci/Documents/Studia/4 rok/Complex systems/Raport 1/generated_data_files/rest/'
               +plik, df.values, fmt='%1.2f')
    

### Output results gathering

In [1]:
generate_file()